# Visualising phase distributions as colour tails

This notebook focuses on **visualising cell-cycle phase distributions** as
“colour tails”: plots where each time point is represented by a bar or tail
coloured by phase percentage.

In this notebook you will:

- load FUCCIphase output tables,
- compute phase percentages over time or conditions,
- generate colour-tail plots suitable for figures and summaries.

Use this notebook if you already have FUCCIphase output and want to produce
compact, visual summaries of phase distributions.


Use this notebook as an example and adapt it to fit your own data.
You will need:

1. An image file
2. Segmentation labels
3. A TrackMate XML file with the tracking results
4. A file with the details of the FUCCI sensor
5. A file with the FUCCI reference intensity curve

In [32]:
import json

import napari
import pandas as pd
import vispy
from aicsimageio import AICSImage
from napari_animation import Animation
from skimage.io import imread

from fucciphase import process_trackmate
from fucciphase.napari import add_trackmate_data_to_viewer, pandas_df_to_napari_tracks
from fucciphase.phase import (
    estimate_percentage_by_subsequence_alignment,
)
from fucciphase.sensor import FUCCISASensor
from fucciphase.utils import postprocess_estimated_percentages

In [35]:
cyan_channel = "MEAN_INTENSITY_CH2"
magenta_channel = "MEAN_INTENSITY_CH1"
regex = r"Track_[0-9]+\\.[a-z]+"
timestep = 0.25  # in hours

reference_file = "../reproducibility/inputs/hacat_fucciphase_reference.csv"
sensor_file = "../example_data/fuccisa_hacat.json"
track_file = "../reproducibility/inputs/merged_linked.ome.xml"
reference_df = pd.read_csv(reference_file)
reference_df.rename(
    columns={"cyan": cyan_channel, "magenta": magenta_channel}, inplace=True
)

In [36]:
with open(sensor_file) as fp:
    sensor_properties = json.load(fp)
sensor = FUCCISASensor(**sensor_properties)

In [40]:
label_file = "../reproducibility/inputs/labels.tif" # you need to provide a label file
labels = imread(label_file)
image = AICSImage("../reproducibility/inputs/downscaled_hacat.ome.tif")
#scale = (image.physical_pixel_sizes.Y, image.physical_pixel_sizes.X)
pixel_size=0.544
scale = (pixel_size,pixel_size)
cyan = image.get_image_dask_data("TYX", C=1)
magenta = image.get_image_dask_data("TYX", C=0)
#actin = image.get_image_dask_data("TYX", C=1)

## Get tracks

In [41]:
track_df = process_trackmate(
    track_file,
    channels=[cyan_channel, magenta_channel],
    sensor=sensor,
    thresholds=[0.1, 0.1],
    generate_unique_tracks=True,
)

C:\Users\SPL_ImageAnalysis-WS\Desktop\fucciphase\src\fucciphase\utils\track_postprocessing.py:352: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["UNIQUE_TRACK_ID"].update(new_track_ids)


In [42]:
track_df["UNIQUE_TRACK_ID"].unique()

array([ 2,  3,  0,  1, 11, 20, 17, 14, 10, 12, 13, 22, 21, 19, 18, 16, 15])

In [43]:
postprocess_estimated_percentages(
    track_df, percentage_column="CELL_CYCLE_PERC", track_id_name="UNIQUE_TRACK_ID"
)
estimate_percentage_by_subsequence_alignment(
    track_df,
    dt=0.25,
    channels=[cyan_channel, magenta_channel],
    reference_data=reference_df,
    track_id_name="UNIQUE_TRACK_ID",
)
postprocess_estimated_percentages(
    track_df, percentage_column="CELL_CYCLE_PERC_DTW", track_id_name="UNIQUE_TRACK_ID"
)

In [44]:
viewer = napari.Viewer()

C:\Users\SPL_ImageAnalysis-WS\anaconda3\envs\fucciphase_run\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'bbii-decon' has already registered a function widget 'bbii deconvolution' which has now been overwritten
  warn(message=warn_message)


In [45]:
add_trackmate_data_to_viewer(
    track_df,
    viewer,
    scale=scale,
    image_data=[cyan, magenta],
    colormaps=["cyan", "magenta"],
    labels=labels,
    cycle_percentage_id="CELL_CYCLE_PERC_DTW",
    label_id_name="MEDIAN_INTENSITY_CH3",
    textkwargs={"size": 16},
)

In [ ]:
viewer.add_image(actin, blending="additive", scale=scale)

In [46]:
colormap = vispy.color.colormap.MatplotlibColormap("cool")

# Normalise percentage

This step is needed to color the tails with respect to the cell cycle.

In [47]:
track_df["CELL_CYCLE_PERC_NORM"] = track_df["CELL_CYCLE_PERC_DTW"] / 100.0

In [48]:
pandas_df_to_napari_tracks(
    track_df,
    viewer,
    unique_track_id_name="UNIQUE_TRACK_ID",
    frame_id_name="FRAME",
    position_x_name="POSITION_X",
    position_y_name="POSITION_Y",
    feature_name="CELL_CYCLE_PERC_NORM",
    colormaps_dict={"CELL_CYCLE_PERC_NORM": colormap},
)

# Add a time stamp

## Color the tails

Choose the color of the track tails according to the 
`CELL_CYCLE_PERC_NORM` by clicking on the respective entry
in the GUI.

More details here:
https://napari.org/0.4.18/howtos/layers/tracks.html#setting-the-track-color-with-properties

In [49]:
old_time = -1


def update_slider(event):
    """Update time stamp."""
    time = viewer.dims.current_step[0]
    global old_time
    if time != old_time:
        old_time = time
    else:
        return
    time = 15.0 * time
    viewer.text_overlay.text = f"{round(time)} min "


viewer.text_overlay.color = "white"
viewer.text_overlay.blending = "translucent_no_depth"
viewer.text_overlay.position = "top_left"
viewer.text_overlay.font_size = 18
viewer.text_overlay.visible = True
viewer.dims.events.current_step.connect(update_slider)

<function __main__.update_slider(event)>

# Save a MP4 video

In [50]:
animation = Animation(viewer)

# start animation on first frame
viewer.dims.current_step = (0, 0, 0)
animation.capture_keyframe()
# last frame
viewer.dims.current_step = (labels.shape[0] - 1, 0, 0)
animation.capture_keyframe(steps=labels.shape[0] - 1)
animation.animate(
    "movie_tails_percentages.mp4", canvas_only=True, fps=4, quality=9, scale_factor=1.0
)

Rendering frames...


 10%|▉         | 12/121 [00:02<00:23,  4.56it/s]


KeyboardInterrupt: 